## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice of this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [139]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [140]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [141]:
df_ibm = df_ibm.loc[data_start:].copy()

In [142]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4180 4140 4160 20


In [143]:
df_ibm_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [144]:
df_ibm_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [145]:
df_ibm_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-01,120.739998,120.739998,122.320000,119.860001,120.349998,5714500,NaN,NaN,120.668,120.434142,122.429981,123.370901,117.965099,123.476928,117.391355,128.961952,115.898011
2021-03-02,120.330002,120.330002,121.900002,120.260002,120.739998,4522200,NaN,NaN,120.694,120.503118,122.358693,123.376258,118.011742,123.465782,117.540453,128.903219,115.814167
2021-03-03,122.360001,122.360001,122.629997,119.980003,120.500000,7396200,NaN,NaN,120.933,120.727868,122.328004,123.751023,118.114978,123.525258,117.930478,128.857510,115.798499
2021-03-04,120.110001,120.110001,123.220001,118.760002,122.000000,8062100,NaN,NaN,120.871,120.855908,122.246395,123.735766,118.006233,123.245385,118.466430,128.782885,115.709906
2021-03-05,122.830002,122.830002,123.750000,120.250000,120.639999,6944900,NaN,NaN,121.255,121.026200,122.267737,124.027120,118.482881,123.470861,118.581540,128.804752,115.730723


#### Apple Inc. Stock train test split

In [146]:
df_aapl = df_aapl.loc[data_start:].copy()

In [147]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4180 4140 4160 20


In [148]:
df_aapl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [149]:
df_aapl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [150]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-01,127.790001,127.790001,127.930000,122.790001,123.750000,115998300.0,NaN,NaN,127.086000,131.304179,131.840745,135.066049,119.105951,141.686518,120.921841,141.633158,122.048332
2021-03-02,125.120003,125.120003,128.720001,125.010002,128.410004,102015300.0,NaN,NaN,126.279000,130.863187,131.790759,133.057281,119.500719,141.519782,120.206592,141.695991,121.885527
2021-03-03,122.059998,122.059998,125.709999,121.839996,124.809998,112430400.0,NaN,NaN,125.401000,130.226757,131.661799,131.818508,118.983492,141.403903,119.049611,141.903656,121.419943
2021-03-04,120.129997,120.129997,123.599998,118.620003,121.750000,177275300.0,NaN,NaN,124.442999,129.546249,131.534979,130.861833,118.024166,141.455994,117.636505,142.187967,120.881991
2021-03-05,121.419998,121.419998,121.940002,117.570000,120.980003,153590400.0,NaN,NaN,123.598000,128.757999,131.402606,128.983132,118.212867,140.625765,116.890234,142.392092,120.413120


#### Amazon Stock train test split

In [151]:
df_amzn = df_amzn.loc[data_start:].copy()

In [152]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4180 4140 4160 20


In [153]:
df_amzn_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [154]:
df_amzn_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [155]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-01,3146.139893,3146.139893,3149.560059,3097.989990,3127.889893,2729100.0,NaN,NaN,3198.671948,3257.981958,3231.172969,3377.903928,3019.439968,3438.263188,3077.700728,3388.284831,3074.061107
2021-03-02,3094.530029,3094.530029,3163.520020,3087.120117,3143.469971,2590000.0,NaN,NaN,3181.229956,3245.564465,3228.244370,3363.978772,2998.481141,3435.193617,3055.935313,3390.001899,3066.486841
2021-03-03,3005.000000,3005.000000,3107.780029,2995.000000,3081.179932,3967200.0,NaN,NaN,3150.865967,3226.814465,3223.622769,3340.306698,2961.425236,3433.835121,3019.793810,3397.236471,3050.009066
2021-03-04,2977.570068,2977.570068,3058.129883,2945.429932,3012.000000,5458700.0,NaN,NaN,3115.799976,3210.066467,3219.141172,3288.394254,2943.205697,3440.735551,2979.397384,3406.123858,3032.158485
2021-03-05,3000.459961,3000.459961,3009.000000,2881.000000,3005.000000,5383400.0,NaN,NaN,3090.855981,3193.539465,3215.026772,3248.791646,2932.920317,3434.845651,2952.233280,3411.962066,3018.091479


#### Alphabet Inc. Stock train test split

In [156]:
df_googl = df_googl.loc[data_start:].copy()

In [157]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4180 4140 4160 20


In [158]:
df_googl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [159]:
df_googl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [160]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-03-01,2069.659912,2069.659912,2076.229980,2034.420044,2048.000000,1303000.0,NaN,NaN,2072.965027,2058.667499,1890.775398,2143.954578,2001.975475,2175.793684,1941.541314,2192.080104,1589.470692
2021-03-02,2064.479980,2064.479980,2094.739990,2056.669922,2064.840088,1277500.0,NaN,NaN,2068.343030,2067.238000,1896.921199,2134.249883,2002.436177,2154.663452,1979.812549,2199.636814,1594.205584
2021-03-03,2011.410034,2011.410034,2075.989990,1994.000000,2051.949951,1692200.0,NaN,NaN,2057.622021,2071.852502,1902.339199,2122.042636,1993.201407,2131.777352,2011.927653,2203.320669,1601.357729
2021-03-04,2033.930054,2033.930054,2074.590088,2005.430054,2014.869995,2457700.0,NaN,NaN,2050.434021,2070.605011,1908.493401,2106.450225,1994.417817,2132.667611,2008.542411,2207.352465,1609.634337
2021-03-05,2097.070068,2097.070068,2106.199951,2027.780029,2061.149902,2649400.0,NaN,NaN,2051.260022,2072.777020,1915.743601,2109.970089,1992.549955,2135.376468,2010.177572,2214.968470,1616.518732


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [161]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

#### IBM stock

In [162]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [163]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [164]:
#print(ibm_ma_mse_loss)

In [165]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [166]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [167]:
print(ibm_ma_mae_loss)

4.3268660545349125


Root Mean Squared Error

In [168]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str], squared=False)

In [169]:
print(ibm_ma_mse_loss)

4.912674690471205


Mean Absolute Percentage Error

In [170]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [171]:
print(ibm_ma_map_loss)

0.033465043233958894


R<sup>2</sup> score

In [172]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [173]:
print(ibm_ma_r2_score)

-0.3157558978028696


In [174]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [175]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [176]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [177]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [178]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [179]:
#print(aapl_ma_mse_loss)

In [180]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [181]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [182]:
print(aapl_ma_mae_loss)

3.9221944999694798


##### Root Mean Squared Error

In [183]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str], squared=False)

In [184]:
print(aapl_ma_rmse_loss)

5.037292833588891


##### Mean Absolute Percentage Error

In [185]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [186]:
print(aapl_ma_map_loss)

0.03241082450831909


R<sup>2</sup> score

In [187]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [188]:
print(aapl_ma_r2_score)

-3.100712294751495


In [189]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [190]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [191]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [192]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [193]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [194]:
#print(amzn_ma_mse_loss)

In [195]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [196]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [197]:
print(amzn_ma_mae_loss)

84.93741577148437


Root Mean Squared Error

In [198]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str], squared=False)

In [199]:
print(amzn_ma_rmse_loss)

113.60399187807732


##### Mean Absolute Percentage Error

In [200]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [201]:
print(amzn_ma_map_loss)

0.0279865370390834


R<sup>2</sup> score

In [202]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [203]:
print(amzn_ma_r2_score)

-3.744705434990844


In [204]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [205]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [206]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [207]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [208]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [209]:
#print(googl_ma_mse_loss)

In [210]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [211]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [212]:
print(googl_ma_mae_loss)

24.373880615234384


Root Mean Squared Error

In [213]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str], squared=False)

In [214]:
print(googl_ma_rmse_loss)

28.73174706163392


##### Mean Absolute Percentage Error

In [215]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [216]:
print(googl_ma_map_loss)

0.011944094825984967


R<sup>2</sup> score

In [217]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [218]:
print(googl_ma_r2_score)

-0.15523429206054784


In [219]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [220]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [221]:
#print(googl_ma_mse_loss)

Volatility

In [222]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.9653995393225406


In [223]:
print(volatility(df_aapl_valid['Adj Close'], n))

0.3256728057221031


In [224]:
print(volatility(df_amzn_valid['Adj Close'], n))

143.1608696590202


In [225]:
print(volatility(df_googl_valid['Adj Close'], n))

37.60974561779712


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.

Now I'll initialize an array of moving average values on validation data, to be used in future model comparison:

In [226]:
df_valid_ma = [df_ibm_valid[:][ma_str], df_aapl_valid[:][ma_str], df_amzn_valid[:][ma_str], df_googl_valid[:][ma_str]]